<a href="https://colab.research.google.com/github/masa1023/neural-network-with-pytorch/blob/main/Neural_Network_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 環境構築

In [ ]:
import torch

In [ ]:
torch.__version__

'2.6.0+cu124'

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

### 2. ネットワークの定義

In [ ]:
import torch.nn as nn

In [ ]:
# 3ノード → 2ノードの全結合層（fully-connected layer）
fc = nn.Linear(3, 2)
fc


Linear(in_features=3, out_features=2, bias=True)

In [ ]:
# 重み
fc.weight

Parameter containing:
tensor([[-0.4153,  0.2511, -0.5048],
        [ 0.4057, -0.0306, -0.2705]], requires_grad=True)

In [ ]:
# バイアス
fc.bias

Parameter containing:
tensor([-0.1155, -0.0279], requires_grad=True)

In [ ]:
# 乱数のシードを固定
torch.manual_seed(0)

In [ ]:
fc = nn.Linear(3, 2)

In [ ]:
fc.weight

Parameter containing:
tensor([[-0.0043,  0.3097, -0.4752],
        [-0.4249, -0.2224,  0.1548]], requires_grad=True)

In [ ]:
fc.bias

Parameter containing:
tensor([-0.0114,  0.4578], requires_grad=True)

### 3. 線形変換

In [ ]:
x = torch.Tensor([1, 2, 3])
x

tensor([1., 2., 3.])

In [ ]:
type(x)

torch.Tensor

In [ ]:
x.dtype

torch.float32

In [ ]:
x.shape

torch.Size([3])

In [ ]:
# 線形変換
# fc.__call__(x)
u = fc(x)
u

tensor([-0.8219,  0.0526], grad_fn=<ViewBackward0>)

### 4. 非線形変換

In [ ]:
import torch.nn.functional as F

In [ ]:
h = F.relu(u)
h

tensor([0.0000, 0.0526], grad_fn=<ReluBackward0>)

### 5. 目的関数

目的関数 = 損失関数 + 正則化項

In [ ]:
# 目標値
t = torch.Tensor([[1], [3]])
t

tensor([[1.],
        [3.]])

In [ ]:
# 予測値
y = torch.Tensor([[2], [4]])
y

tensor([[2.],
        [4.]])

In [ ]:
# 平均二乗誤差
F.mse_loss(y, t)

tensor(1.)

### 6. nn と F の使い分け
- `nn`: `torch.nn` -> パラメータを持つ (weightやbiasなど)
- `F`: `torch.nn.functional` -> パラメータを持たない

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
# データセットの読み込み
breast_cancer = load_breast_cancer()

In [ ]:
# 入力と目標値を抽出
x = breast_cancer.data
t = breast_cancer.target

In [ ]:
x.shape

(569, 30)

In [ ]:
t.shape

(569,)

In [ ]:
type(x), type(t)

(numpy.ndarray, numpy.ndarray)

In [ ]:
# ndarray -> Tensor
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [ ]:
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [ ]:
x.dtype, t.dtype

(torch.float32, torch.int64)

In [ ]:
x.shape, t.shape

(torch.Size([569, 30]), torch.Size([569]))

- 入力値: float32
- 目標値
  - 回帰: float32
  - 分類
    - 二値分類: float32
    - 多値分類: int64

### 2. DataLoader

In [ ]:
# 入力値と目標値を dataset としてまとめる
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [ ]:
type(dataset)

torch.utils.data.dataset.TensorDataset

In [ ]:
dataset[0]

(tensor([1.7990e+01, 1.0380e+01, 1.2280e+02, 1.0010e+03, 1.1840e-01, 2.7760e-01,
         3.0010e-01, 1.4710e-01, 2.4190e-01, 7.8710e-02, 1.0950e+00, 9.0530e-01,
         8.5890e+00, 1.5340e+02, 6.3990e-03, 4.9040e-02, 5.3730e-02, 1.5870e-02,
         3.0030e-02, 6.1930e-03, 2.5380e+01, 1.7330e+01, 1.8460e+02, 2.0190e+03,
         1.6220e-01, 6.6560e-01, 7.1190e-01, 2.6540e-01, 4.6010e-01, 1.1890e-01]),
 tensor(0))

In [ ]:
dataset[0][0]

tensor([1.7990e+01, 1.0380e+01, 1.2280e+02, 1.0010e+03, 1.1840e-01, 2.7760e-01,
        3.0010e-01, 1.4710e-01, 2.4190e-01, 7.8710e-02, 1.0950e+00, 9.0530e-01,
        8.5890e+00, 1.5340e+02, 6.3990e-03, 4.9040e-02, 5.3730e-02, 1.5870e-02,
        3.0030e-02, 6.1930e-03, 2.5380e+01, 1.7330e+01, 1.8460e+02, 2.0190e+03,
        1.6220e-01, 6.6560e-01, 7.1190e-01, 2.6540e-01, 4.6010e-01, 1.1890e-01])

In [ ]:
dataset[0][1]

tensor(0)

- 訓練データ: ネットワークのパラメータの最適化(w, b)
- 検証データ: ネットワークのハイパーパラメータの最適化(e.g. learning rate, layer, node)
- テストデータ: 訓練済みネットワークの評価

In [ ]:
# 各データセットのサンプル数を決定
# train : val : test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

n_train, n_val, n_test

(341, 113, 115)

In [ ]:
# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

In [ ]:
# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [ ]:
# サンプル数を確認
len(train), len(val), len(test)

(341, 113, 115)

In [ ]:
# バッチサイズ
batch_size = 10

In [ ]:
# DataLoaderの定義
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size)

In [ ]:
x, t = next(iter(train_loader))

In [ ]:
x.shape

torch.Size([10, 30])

In [ ]:
t.shape

torch.Size([10])

In [ ]:
x, t = train[0]

In [ ]:
x.shape

torch.Size([30])

In [ ]:
t

tensor(1)

In [ ]:
x.unsqueeze(0).shape

torch.Size([1, 30])

In [ ]:
# 乱数シードの固定
torch.manual_seed(0)

In [ ]:
# 全結合層の定義
fc1 = nn.Linear(30, 10)
fc2 = nn.Linear(10, 2)

In [ ]:
fc1.weight

Parameter containing:
tensor([[-0.0014,  0.0979, -0.1503, -0.1344, -0.0703,  0.0490, -0.0036,  0.1448,
         -0.0162,  0.0483, -0.0552, -0.0359, -0.1744, -0.1209, -0.0753,  0.0068,
          0.0722,  0.1095, -0.1238, -0.0795,  0.0663,  0.1516, -0.0376,  0.1366,
         -0.0294,  0.0193,  0.1653, -0.1694, -0.1149, -0.0462],
        [-0.0712,  0.1577, -0.1183, -0.0840, -0.1276, -0.1710, -0.1066,  0.1569,
          0.0815,  0.0885,  0.0096, -0.0936,  0.0309, -0.1705, -0.1319, -0.0941,
          0.1152,  0.1070, -0.0810, -0.0066,  0.1168,  0.1815,  0.0725,  0.0247,
          0.1224, -0.1075,  0.0340, -0.1416, -0.1265, -0.0943],
        [ 0.0826,  0.0734, -0.1081,  0.0552,  0.1002, -0.0230,  0.0070,  0.0423,
          0.1133,  0.1753, -0.1407, -0.0669,  0.0718,  0.1513,  0.1589,  0.1611,
          0.0363, -0.1588,  0.0168, -0.1142, -0.1702,  0.1622,  0.1388, -0.1821,
          0.0342, -0.0308, -0.0300, -0.0836,  0.0702, -0.1081],
        [ 0.0669,  0.0923,  0.1307,  0.0683, -0.1807, -0.

# 順伝播
- 線形結合 fc1
- 非線形変換 ReLU
- 線形変換 fc2
- 非線形変換 Softmax

In [ ]:
x.unsqueeze(0).shape

torch.Size([1, 30])

In [ ]:
h = fc1(x.unsqueeze(0))

In [ ]:
h.shape

torch.Size([1, 10])

In [ ]:
h = F.relu(h)
h

tensor([[  0.1830,   0.0000,   0.0000, 142.6534,  51.1358,  95.4793,   0.0000,
          87.3314,  94.4835,  10.6316]], grad_fn=<ReluBackward0>)

In [ ]:
h = fc2(h)
h

tensor([[22.3666, 57.7020]], grad_fn=<AddmmBackward0>)

In [ ]:
h = F.softmax(h)
h

<ipython-input-58-3e9aa2b0c6ea>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  h = F.softmax(h)


tensor([[4.5087e-16, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 10)
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.fc2(h)
        # h = F.softmax(h)
        return h

In [ ]:
# 乱数のシードの固定
torch.manual_seed(0)

# インスタンス化
net = Net()
net

Net(
  (fc1): Linear(in_features=30, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
)

In [ ]:
# net.forward(x.unsqueeze(0))
net(x.unsqueeze(0))

tensor([[22.3666, 57.7020]], grad_fn=<AddmmBackward0>)

### 3. 損失関数

交差エントロピー

$$
L = - \dfrac{1}{N} \sum_{n=1}^{N} \sum_{c=1}^{C} t_{nc} \log y_{nc}
$$

`F.cross_entropy`

### 4. 最適化手法

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [ ]:
# for parameter in net.parameters():
#   print(parameter)
print(*net.parameters())

Parameter containing:
tensor([[-0.0014,  0.0979, -0.1503, -0.1344, -0.0703,  0.0490, -0.0036,  0.1448,
         -0.0162,  0.0483, -0.0552, -0.0359, -0.1744, -0.1209, -0.0753,  0.0068,
          0.0722,  0.1095, -0.1238, -0.0795,  0.0663,  0.1516, -0.0376,  0.1366,
         -0.0294,  0.0193,  0.1653, -0.1694, -0.1149, -0.0462],
        [-0.0712,  0.1577, -0.1183, -0.0840, -0.1276, -0.1710, -0.1066,  0.1569,
          0.0815,  0.0885,  0.0096, -0.0936,  0.0309, -0.1705, -0.1319, -0.0941,
          0.1152,  0.1070, -0.0810, -0.0066,  0.1168,  0.1815,  0.0725,  0.0247,
          0.1224, -0.1075,  0.0340, -0.1416, -0.1265, -0.0943],
        [ 0.0826,  0.0734, -0.1081,  0.0552,  0.1002, -0.0230,  0.0070,  0.0423,
          0.1133,  0.1753, -0.1407, -0.0669,  0.0718,  0.1513,  0.1589,  0.1611,
          0.0363, -0.1588,  0.0168, -0.1142, -0.1702,  0.1622,  0.1388, -0.1821,
          0.0342, -0.0308, -0.0300, -0.0836,  0.0702, -0.1081],
        [ 0.0669,  0.0923,  0.1307,  0.0683, -0.1807, -0.

### 5. ネットワークの訓練の流れを確認

In [ ]:
batch = next(iter(train_loader))
# Step 1
x, t = batch

In [ ]:
# Step 2
y = net(x)

In [ ]:
# Step 3
loss = F.cross_entropy(y, t)

In [ ]:
net.fc1.weight.grad
# net.fc1.bias.grad

In [ ]:
# Step 4
optimizer.zero_grad() # 勾配の初期化
loss.backward() # 勾配の算出

In [ ]:
net.fc1.weight.grad
# net.fc1.bias.grad
# net.fc2.weight.grad
# net.fc2.bias.grad

tensor([[ 2.3378e+00,  3.4356e+00,  1.5772e+01,  1.2823e+02,  1.6362e-02,
          2.9557e-02,  3.4264e-02,  1.4788e-02,  3.0445e-02,  1.0216e-02,
          6.6737e-02,  2.2061e-01,  5.5411e-01,  7.8438e+00,  1.1115e-03,
          9.1272e-03,  1.1084e-02,  2.6349e-03,  3.5681e-03,  1.0354e-03,
          2.6589e+00,  5.0072e+00,  1.8492e+01,  1.6547e+02,  2.3532e-02,
          8.8433e-02,  1.0870e-01,  3.0511e-02,  5.3197e-02,  1.8125e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  

In [ ]:
# Step 5
optimizer.step() # パラメータの更新

In [ ]:
net.fc1.weight

Parameter containing:
tensor([[-2.4745e-02,  6.3585e-02, -3.0799e-01, -1.4166e+00, -7.0483e-02,
          4.8663e-02, -3.9600e-03,  1.4461e-01, -1.6507e-02,  4.8209e-02,
         -5.5844e-02, -3.8094e-02, -1.7996e-01, -1.9935e-01, -7.5272e-02,
          6.6719e-03,  7.2067e-02,  1.0952e-01, -1.2381e-01, -7.9515e-02,
          3.9725e-02,  1.0154e-01, -2.2250e-01, -1.5181e+00, -2.9663e-02,
          1.8435e-02,  1.6423e-01, -1.6967e-01, -1.1547e-01, -4.6403e-02],
        [-7.1167e-02,  1.5774e-01, -1.1834e-01, -8.4045e-02, -1.2755e-01,
         -1.7099e-01, -1.0658e-01,  1.5694e-01,  8.1468e-02,  8.8489e-02,
          9.6019e-03, -9.3603e-02,  3.0889e-02, -1.7047e-01, -1.3192e-01,
         -9.4122e-02,  1.1519e-01,  1.0705e-01, -8.0971e-02, -6.5877e-03,
          1.1677e-01,  1.8150e-01,  7.2460e-02,  2.4664e-02,  1.2241e-01,
         -1.0750e-01,  3.4022e-02, -1.4155e-01, -1.2654e-01, -9.4315e-02],
        [ 8.2610e-02,  7.3424e-02, -1.0815e-01,  5.5157e-02,  1.0023e-01,
         -2.30

In [ ]:
torch.cuda.is_available()

True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
# 指定したデバイスへネットワークの転送
net.to(device)

Net(
  (fc1): Linear(in_features=30, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
)

In [ ]:
x = x.to(device)
y = y.to(device)

In [ ]:
y = net(x)
y

tensor([[ 148.3002, -284.6127],
        [ 163.9481, -312.1373],
        [ 493.4534, -945.2295],
        [ 430.1196, -824.0096],
        [ 185.5045, -355.8210],
        [ 170.7496, -327.5804],
        [ 187.2742, -359.2082],
        [ 145.2411, -278.7576],
        [ 156.5140, -300.3338],
        [ 121.6984, -232.1702]], device='cuda:0', grad_fn=<AddmmBackward0>)

### 6. ネットワークを訓練

In [ ]:
# エポック数
max_epoch = 1 # 1 epoch = 34 iteration * 10 batch

In [ ]:
torch.manual_seed(0)
net = Net().to(device)

In [ ]:
# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [ ]:
# 訓練のループ
for epoch in range(max_epoch):
  for batch in train_loader:
      # Step 1
      x, t = batch
      x = x.to(device)
      t = t.to(device)
      # Step 2
      y = net(x)
      # Step 3
      loss = F.cross_entropy(y, t)
      print(f'loss: {loss}')
      # Step 4
      optimizer.zero_grad()
      loss.backward()
      # Step 5
      optimizer.step()

loss: 33.399085998535156
loss: 15349.416015625
loss: 0.693260669708252
loss: 0.6789408326148987
loss: 0.6570119261741638
loss: 0.6791458129882812
loss: 0.6593756675720215
loss: 0.6770837306976318
loss: 0.7460513710975647
loss: 0.6792094111442566
loss: 0.6213985681533813
loss: 0.6232878565788269
loss: 0.643330454826355
loss: 0.7069395780563354
loss: 0.6739681363105774
loss: 0.7380276322364807
loss: 0.6462857723236084
loss: 0.6119552254676819
loss: 0.7443729639053345
loss: 0.6421698331832886
loss: 0.5709103345870972
loss: 0.7135142087936401
loss: 0.7115662693977356
loss: 0.7464044094085693
loss: 0.5425654053688049
loss: 0.714184045791626
loss: 0.7121732831001282
loss: 0.7475570440292358
loss: 0.6736332178115845
loss: 0.6735744476318359
loss: 0.7075648307800293
loss: 0.6738204956054688
loss: 0.6737439632415771
loss: 0.6405441761016846


### 7. 評価指標の算出

In [ ]:
x, t = next(iter(train_loader))

In [ ]:
net.cpu()

Net(
  (fc1): Linear(in_features=30, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
)

In [ ]:
y = net(x)
y

tensor([[-0.4444, -0.0895],
        [-0.4444, -0.0895],
        [-0.4444, -0.0895],
        [-0.4444, -0.0895],
        [-0.4444, -0.0895],
        [-0.4444, -0.0895],
        [-0.4444, -0.0895],
        [-0.4444, -0.0895],
        [-0.4444, -0.0895],
        [-0.4444, -0.0895]], grad_fn=<AddmmBackward0>)

In [ ]:
t

tensor([0, 1, 1, 0, 0, 1, 0, 0, 1, 0])

In [ ]:
# torch.argmax(F.softmax(y), dim=1)

<ipython-input-90-f6a2b27dc2d6>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.argmax(F.softmax(y), dim=1)


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
y_label = torch.argmax(y, dim=1)
y_label

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
y_label == t

tensor([False,  True,  True, False, False,  True, False, False,  True, False])

In [ ]:
(y_label == t).sum() / len(t)

tensor(0.4000)

### 8. 正解率を追加

In [ ]:
# エポック数
max_epoch = 1 # 1 epoch = 34 iteration * 10 batch
net = Net().to(device)

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [ ]:
# 訓練のループ
for epoch in range(max_epoch):
  for batch in train_loader:
      # Step 1
      x, t = batch
      x = x.to(device)
      t = t.to(device)

      # Step 2
      y = net(x)

      # 正解率 (accuracy) を追加
      y_label = torch.argmax(y, dim=1)
      accuracy = (y_label == t).sum() / len(t)
      print(f'accuracy: {accuracy:.2f}')

      # Step 3
      loss = F.cross_entropy(y, t)
      # print(f'loss: {loss}')

      # Step 4
      optimizer.zero_grad()
      loss.backward()

      # Step 5
      optimizer.step()

accuracy: 0.30
accuracy: 0.70
accuracy: 0.60
accuracy: 0.80
accuracy: 0.70
accuracy: 0.60
accuracy: 0.80
accuracy: 0.70
accuracy: 0.60
accuracy: 0.60
accuracy: 0.60
accuracy: 0.60
accuracy: 0.50
accuracy: 0.20
accuracy: 0.80
accuracy: 0.30
accuracy: 0.60
accuracy: 0.50
accuracy: 0.70
accuracy: 0.80
accuracy: 0.90
accuracy: 0.80
accuracy: 0.20
accuracy: 0.70
accuracy: 0.60
accuracy: 0.70
accuracy: 0.50
accuracy: 0.80
accuracy: 0.80
accuracy: 0.20
accuracy: 0.60
accuracy: 0.70
accuracy: 0.70
accuracy: 0.50


### 9. 訓練後の正解率を検証
with torch.no_grad # 訓練しない → 勾配の計算が不要なため

In [ ]:
# val_loader
def calc_accuracy(data_loader):
  with torch.no_grad():
    total = 0
    correct = 0.0

    for batch in data_loader:
      x, t = batch
      x = x.to(device)
      t = t.to(device)
      y = net(x)

      y_label = torch.argmax(y, dim=1)
      total += len(y)
      correct += (y_label == t).sum()

    accuracy = correct / total
  return accuracy

In [ ]:
calc_accuracy(val_loader)

tensor(0.6637, device='cuda:0')

In [ ]:
calc_accuracy(test_loader)

tensor(0.6087, device='cuda:0')